
# BADFit
This is a tutorial notebook for __[BADFit](https://github.com/samlaihei/BADFit)__, a code for fitting black hole accretion discs with Sherpa/XSpec. The key features of this code are:
-  PLACEHOLDER

### Installation
The recommended installation method is to use conda virtual environments. This section will guide the user through the basic installation steps which should work for most machines. Navigate to your designated installation directory and run in the terminal,
```
git clone https://github.com/samlaihei/PyQSpecFit.git
cd PyQSpecFit
conda env create -f environment.yml
```
Blah


### Features 
-  PLACEHOLDER

### Authors
__[Samuel Lai](https://orcid.org/0000-0001-9372-4611)__; samuel.lai(at)anu.edu.au (Australian National University)

### Acknowledgements
If you found this code useful in any capacity, we would appreciate it if you could cite us. Please refer to the __[README.md](https://github.com/samlaihei/BADFit)__ in the repository for the most up-to-date citation.


### General Usage Procedure
1. PLACEHOLDER


## Getting Started

